In [3]:
import os # for os.path
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [6]:
## Langsmith Tracking and Tracing

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [5]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="o1-mini")
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x10d51aa80> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10d5c3350> root_client=<openai.OpenAI object at 0x10c8d67e0> root_async_client=<openai.AsyncOpenAI object at 0x10d36e690> model_name='o1-mini' temperature=1.0 model_kwargs={} openai_api_key=SecretStr('**********')


In [8]:
result=llm.invoke("What is the capital of France?")
print(result.content)


The capital of France is **Paris**.


In [12]:
from langchain_groq import ChatGroq

model=ChatGroq(model="qwen-qwq-32b")

model.invoke("What is the capital of France?")


AIMessage(content='\n<think>\nOkay, the user is asking, "What is the capital of France?" Let me think about how to approach this.\n\nFirst, I need to confirm the answer. The capital of France is Paris. But wait, maybe I should double-check to be sure. Sometimes there can be confusion, especially with countries that have historical changes. For example, some countries have had their capitals moved due to political reasons. But France\'s capital has been Paris for a long time. \n\nI should also consider if there\'s any recent change that I\'m not aware of. A quick mental recap: France is a well-known country, and Paris is its most famous city. It\'s the political, cultural, and economic center. The government is there, including the presidency and parliament. \n\nHmm, could there be any trick in the question? Maybe they\'re looking for something else, but I don\'t think so. The question is straightforward. The user might be a student needing the answer for homework, or someone learning b

In [44]:
### Prompt Engineering

from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert AI Engineer. Provide me answer based on the question"),
        ("user", "{input}")
    ]
)

prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer. Provide me answer based on the question'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [23]:
from langchain_groq import ChatGroq
model1=ChatGroq(model="gemma2-9b-it")

model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10dda2ae0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1189c6a20>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [45]:
### Chaining 

chain=prompt|model
chain

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer. Provide me answer based on the question'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10dda2ae0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1189c6a20>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [28]:
response= chain.invoke({"input": "Can you tell me something about the Langsmith?"})

print(response.content)

You're asking about **Langsmith**, which is a fantastic project! 

Langsmith is an open-source platform designed to make it easier to build and deploy large language models (LLMs). Think of it as a toolbox packed with helpful utilities specifically for working with LLMs. 

Here are some key things to know about Langsmith:

* **Democratizing LLM Development:** Langsmith aims to lower the barrier to entry for building and experimenting with LLMs. It provides a streamlined environment that doesn't require extensive technical expertise.
* **Python-First Approach:**  It's built with Python in mind, making it accessible to a wide range of developers.

* **Modular Design:** Langsmith is modular, meaning you can easily integrate different components and customize your workflow.

* **Streamlined Experimentation:** It simplifies the process of training, evaluating, and fine-tuning LLMs. You can quickly iterate on your models and see the results.

* **Community-Driven:** Langsmith is an open-sour

In [35]:
### Output Parsers

from langchain_core.output_parsers import StrOutputParser

output_parser=StrOutputParser()

chain= prompt|model|output_parser

response= chain.invoke({"input": "can you tell me something about the Langsmith?"})

print(response)



Ah, you're asking about Langsmith! It's a fascinating project. Let me tell you what I know:

**Langsmith** is an open-source platform developed by the Gemma team at Google DeepMind. It's designed to streamline the process of building and deploying large language models (LLMs). Think of it as a toolbox specifically for working with LLMs, offering a range of features and tools to make the process more efficient and accessible.

Here are some key things to know about Langsmith:

* **Focus on Efficiency:** Langsmith emphasizes building and using LLMs efficiently. It offers tools for fine-tuning existing models, allowing you to customize them for specific tasks without needing to train them from scratch, which can be incredibly resource-intensive.
* **Open-Source and Collaborative:** As an open-source project, Langsmith thrives on community contributions. This means you can benefit from the work of others, share your own tools and improvements, and collaborate with a global network of resea

In [46]:
### Output Parser using JSON

from langchain_core.output_parsers import JsonOutputParser

output_parser=JsonOutputParser()
output_parser.get_format_instructions()



'Return a JSON object.'

In [47]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

output_parser=JsonOutputParser()

prompt= PromptTemplate(
    input_variables=["query"],
    template="answer the query \n {format_instruction} \n {query}\n",
    partial_variables={"format_instruction": output_parser.get_format_instructions()},

)

In [48]:
prompt

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instruction': 'Return a JSON object.'}, template='answer the query \n {format_instruction} \n {query}\n')

In [50]:
chain= prompt|model|output_parser
response= chain.invoke({"query": "can you tell me something about the Langsmith?"})
print(response)

{'name': 'Langsmith', 'description': 'Langsmith is an open-source platform for developing and deploying AI assistants. ', 'features': ['Allows users to build, train, and fine-tune their own language models.', 'Provides a library of pre-trained models for various tasks.', 'Offers tools for evaluating and monitoring model performance.', 'Enables seamless integration with other AI tools and services.'], 'creator': 'Hugging Face', 'website': 'https://www.huggingface.co/blog/introducing-langsmith'}


In [52]:
## Assignment  -- chatPromptTemplate

### Prompt Engineering

from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages(
    [
    ("system","You are an expert AI Engineer.Provide the response in json.Provide me answer based on the question"),
        ("user","{input}")
    ]
)

prompt


ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer.Provide the response in json.Provide me answer based on the question'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [53]:
chain=prompt|model|output_parser
response=chain.invoke({"input":"Can you tell me about Langsmith?"})
print(response)

{'response': "Langsmith is an open-source platform developed by the Hugging Face community for building and deploying large language models (LLMs). It simplifies the process of fine-tuning and using pre-trained LLMs for various downstream tasks, such as text generation, summarization, and question answering.\n\n  Here are some key features of Langsmith:\n\n  * **User-friendly Interface:** Langsmith provides a web-based interface that makes it easy to interact with LLMs, even for users without extensive coding experience.\n\n  * **Fine-tuning Capabilities:** It allows users to fine-tune pre-trained LLMs on their own datasets, enabling them to customize the model's performance for specific applications.\n\n  * **Model Management:** Langsmith offers tools for managing and versioning different LLM models, facilitating collaboration and reproducibility.\n\n  * **Pipeline Creation:** Users can build and deploy pipelines that chain together multiple LLMs and other tools to create complex work

In [58]:
### OutputParser using XML

from langchain_core.output_parsers import XMLOutputParser
output_parser=XMLOutputParser()
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are an expert AI Engineer.Provide the response in XML markup format with <output>  tags.Provide me answer based on the question"),
        ("user","{query}")
    ]
)
prompt

ChatPromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer.Provide the response in XML markup format with <output>  tags.Provide me answer based on the question'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})])

In [63]:
chain=prompt|model|output_parser
response=chain.invoke({"query":"Can you tell me about Langsmith?"})
print(response)

{'output': "\n\nLangsmith is an open-source platform developed by Replicate that simplifies the process of training and deploying large language models (LLMs). \n\nHere are some key features of Langsmith:\n\n* **User-Friendly Interface:** Langsmith provides a web-based interface that makes it accessible to both developers and non-developers.\n* **Streamlined Training Process:** It offers tools and workflows to streamline the training of LLMs, including data preparation, model selection, and hyperparameter tuning.\n* **Model Deployment:** Langsmith enables easy deployment of trained LLMs as APIs, allowing developers to integrate them into their applications.\n* **Community-Driven:** As an open-source platform, Langsmith benefits from a vibrant community of contributors who contribute to its development and support.\n* **Integration with Replicate:** Langsmith is tightly integrated with Replicate, a platform for hosting and running machine learning models.\n\n**Benefits of using Langsmit

In [64]:
##output parser
#from langchain_core.output_parsers import XMLOutputParser
from langchain.output_parsers.xml import XMLOutputParser

# XML Output Parser
output_parser = XMLOutputParser()

# Prompt that instructs the model to return XML
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Respond in this XML format: <response><answer>Your answer here</answer></response>"),
    ("human", "{input}")
])

# Build the chain
chain = prompt | model

# Run the chain
#response = chain.invoke({"input": "What is LangChain?"})

raw_output =chain.invoke({"input": "What is LangChain?"})

# Print result
print(raw_output)


content='<response><answer>LangChain is an open-source framework designed to simplify the development of applications powered by large language models (LLMs). It provides a set of tools and building blocks to help developers create, manage, and interact with LLMs in a more efficient and flexible way. \n\nHere are some key features of LangChain:\n\n- **Chain creation:** Allows you to build chains of LLMs and other components (like search engines or databases) to perform complex tasks.\n- **Memory management:** Provides tools for managing and persisting the context of conversations with LLMs.\n- **Prompt templates:** Offers a way to define and reuse prompts for different use cases.\n- **Agents:** Enables the creation of autonomous agents that can interact with the world through APIs and other tools.\n\nOverall, LangChain aims to make it easier for developers to leverage the power of LLMs in a wide range of applications.</answer></response>\n' additional_kwargs={} response_metadata={'toke

In [65]:
## With Pydantic
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

model = ChatOpenAI(temperature=0.7)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

{'setup': "Why couldn't the bicycle find its way home?",
 'punchline': 'Because it lost its bearings!'}

In [66]:
### Without Pydantic
joke_query = "Tell me a joke ."

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

{'joke': "Why couldn't the bicycle stand up by itself? It was two tired!"}

In [67]:
from langchain_core.output_parsers import XMLOutputParser
from langchain_core.prompts import PromptTemplate


actor_query = "Generate the shortened filmography for Tom Hanks."

output = model.invoke(
    f"""{actor_query}
Please enclose the movies in <movie></movie> tags"""
)

print(output.content)

<movie>Big</movie>
<movie>Forrest Gump</movie>
<movie>Cast Away</movie>
<movie>Saving Private Ryan</movie>
<movie>The Green Mile</movie>
<movie>Apollo 13</movie>
<movie>Philadelphia</movie>
<movie>Sully</movie>
<movie>Toy Story</movie>


In [68]:
from langchain.output_parsers import YamlOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


model = ChatOpenAI(temperature=0.5)

# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = YamlOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": joke_query})

Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')

### Assisgment:
Create a simple assistant that uses any LLM and should be pydantic, when we ask about any product it should give you two information product Name, product details tentative price in USD (integer). use chat Prompt Template.


In [72]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

# Pydantic class for the output
class ProductDetails(BaseModel):
    product_name: str = Field(description="Name of the product")
    price: int = Field(description="Price in USD")
    details: str = Field(description="Details of the product")


# Output Parser

parser= JsonOutputParser(pydantic_object=ProductDetails)

# Chat Prompt Template
prompt= ChatPromptTemplate.from_messages(
    [
        ("system", "You are an Product Expert. Provide the product details in JSON format, including product name, price, and details."),
        ("user", "{input}")

    ]
)

# Model
model= ChatOpenAI(model="gpt-4o")

# Chaining
chain= prompt|model|parser

# Invoke the chain
response= chain.invoke({"input": "provide the details about the audi a6"})
print(response)


{'product_name': 'Audi A6', 'price': 'Starting at $55,900', 'details': {'engine': '2.0L TFSI I4 engine', 'horsepower': '261 hp', 'transmission': '7-speed S tronic automatic', 'drivetrain': 'quattro all-wheel drive', 'fuel_economy': {'city': '24 mpg', 'highway': '31 mpg'}, 'interior_features': ['Leather seating surfaces', '3-zone automatic climate control', 'Audi virtual cockpit'], 'technology': ['MMI touch response with dual center displays', 'Audi smartphone interface', 'Bang & Olufsen 3D sound system'], 'safety_features': ['Audi pre sense basic and city', 'Lane departure warning', 'Rear view camera'], 'dimensions': {'length': '194.2 inches', 'wheelbase': '115.1 inches', 'cargo_capacity': '13.7 cubic feet'}, 'warranty': '4-year/50,000-mile limited warranty'}}


In [73]:
# Invoke the chain
response= chain.invoke({"input": "provide the details about iphone 16 pro max"})
print(response)

{'product_name': 'iPhone 16 Pro Max', 'price': '1399 USD', 'details': {'display': '6.9-inch Super Retina XDR', 'processor': 'A19 Bionic chip', 'storage_options': ['256GB', '512GB', '1TB', '2TB'], 'camera_system': 'Triple 48MP Ultra-Wide, Wide, and Telephoto cameras', 'battery_life': 'Up to 28 hours talk time', 'design': 'Ceramic Shield front, Textured matte glass back, and Surgical-grade stainless steel frame', 'operating_system': 'iOS 18', 'features': ['5G capable', 'ProMotion technology with adaptive refresh rates up to 120Hz', 'Face ID', 'MagSafe technology', 'Improved Night mode and Smart HDR'], 'color_options': ['Graphite', 'Gold', 'Silver', 'Sierra Blue']}}
